In [1]:
from test import *

In [2]:
# Constants
N_word = 300
B_word = 42
USE_SMALL = True
GPU = True
BATCH_SIZE = 15
TRAIN_AGG = TRAIN_SEL = TRAIN_COND = True
TRAIN_ENTRY = (TRAIN_AGG, TRAIN_SEL, TRAIN_COND)
learning_rate = 1e-3

In [3]:
sql_data, table_data, val_sql_data, val_table_data, test_sql_data, test_table_data, TRAIN_DB, DEV_DB, TEST_DB = load_dataset(0, use_small=USE_SMALL)
word_emb = load_word_emb('glove/glove.%dB.%dd.txt'%(B_word,N_word), load_used=True, use_small=USE_SMALL)

Loading from original dataset
Loading data from data/train_tok.jsonl
Loading data from data/train_tok.tables.jsonl
Loading data from data/dev_tok.jsonl
Loading data from data/dev_tok.tables.jsonl
Loading data from data/test_tok.jsonl
Loading data from data/test_tok.tables.jsonl
Load used word embedding


In [4]:
model = SQLNet(word_emb, N_word=N_word, use_ca=True, gpu=GPU, trainable_emb = True)
agg_m = 'saved_model/agg.agg_model'
sel_m = 'saved_model/sel.sel_model'
cond_m = 'saved_model/cond.cond_model'
print "Loading from %s"%agg_m
model.agg_pred.load_state_dict(torch.load(agg_m))
print "Loading from %s"%sel_m
model.sel_pred.load_state_dict(torch.load(sel_m))
print "Loading from %s"%cond_m
model.cond_pred.load_state_dict(torch.load(cond_m))

Using trainable embedding
Using trainable embedding
Using trainable embedding
Using column attention on aggregator predicting
Using column attention on selection predicting
Using column attention on where predicting
Loading from saved_model/agg.agg_model
Loading from saved_model/sel.sel_model
Loading from saved_model/cond.cond_model


In [10]:
perm = list(range(len(test_sql_data)))
dummy_res = to_batch_seq(test_sql_data, test_table_data, perm, 0, 1, ret_vis_data=True)
dummy_query = test_sql_data[0]

In [11]:
def query_to_seq(query, data):
    conds = query["sql"]["conds"]
    
    q_seq = [query["question_tok"], dummy_res[0][0]]
    col_seq = [data["header_tok"], dummy_res[1][0]]
    col_num = [len(data["header"]), dummy_res[2][0]]
    ans_seq = [(query["sql"]["agg"], query["sql"]["sel"], len(conds), tuple(x[0] for x in conds), tuple(x[1] for x in conds)), dummy_res[3][0]]
    query_seq = [query["query_tok"], dummy_res[4][0]]
    gt_cond_seq = [query["sql"]["conds"], dummy_res[5][0]]
    vis_seq = [(query["question"], data["header"], query["query"]), dummy_res[6][0]]
    
    return q_seq, col_seq, col_num, ans_seq, query_seq, gt_cond_seq, vis_seq


def query_to_batch(query):
    query_gt = [query["sql"], dummy_query["sql"]]
    table_ids = [query["table_id"], dummy_query["table_id"]]
    return query_gt, table_ids

In [37]:
pred_entry = (True, True, True)
model.eval()

agg_funcs = [None, "max", "min", "count"]
ops = ["=", ">", "<"]
def pred(model, query, data):
    q_seq, col_seq, col_num, ans_seq, query_seq, gt_cond_seq, raw_data = query_to_seq(query, data)
    raw_q_seq = [x[0] for x in raw_data]
    raw_col_seq = [x[1] for x in raw_data]
    gt_sel_seq = [x[1] for x in ans_seq]
    score = model.forward(q_seq, col_seq, col_num, pred_entry, gt_sel = gt_sel_seq)
    pred_queries = model.gen_query(score, q_seq, col_seq, raw_q_seq, raw_col_seq, pred_entry, verbose=True)
    pred_q = pred_queries[0]
    pred_q["agg"] = agg_funcs[pred_q["agg"]]
    pred_q["sel"] = data["header"][pred_q["sel"]]
    for cond in pred_q["conds"]:
        cond[0] = data["header"][cond[0]]
        cond[1] = ops[cond[1]]
    return pred_q




In [38]:
# Test with existing data
idx = 15
q = test_sql_data[idx]
print(q["question"])
data = test_table_data[q["table_id"]]
pred(model, q, data)

List the # for ships commissioned on december 18, 1965.


{'agg': 'count',
 'conds': [[u'Status', '=', u'december 18, 1965']],
 'sel': u'#'}

In [34]:
q

{u'phase': 1,
 u'query': u'SELECT # WHERE commissioned EQL december 18, 1965',
 u'query_tok': [u'SELECT',
  u'#',
  u'WHERE',
  u'commissioned',
  u'EQL',
  u'december',
  u'18',
  u',',
  u'1965'],
 u'query_tok_space': [u' ', u' ', u' ', u' ', u' ', u' ', u'', u' ', u''],
 u'question': u'List the # for ships commissioned on december 18, 1965.',
 u'question_tok': [u'list',
  u'the',
  u'#',
  u'for',
  u'ships',
  u'commissioned',
  u'on',
  u'december',
  u'18',
  u',',
  u'1965',
  u'.'],
 u'question_tok_space': [u' ',
  u' ',
  u' ',
  u' ',
  u' ',
  u' ',
  u' ',
  u' ',
  u'',
  u' ',
  u'',
  u''],
 u'sql': {u'agg': 0, u'conds': [[4, 0, u'December 18, 1965']], u'sel': 0},
 u'table_id': u'1-1014206-2'}